In [3]:
%matplotlib inline
from re import DEBUG
from turtle import color
import numpy as np
import pandas as pd
from prettytable import PrettyTable
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge,RidgeCV, LassoCV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, StackingRegressor, VotingRegressor, AdaBoostRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from warnings import simplefilter
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
from pathlib import WindowsPath
import time
from collections import defaultdict
import re
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
import datetime
import pickle
from collections import Counter
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

DEBUGGING: bool = False
NUM_EVENT_PER_POS = 100
MAP_SET_PADS_TO_TRIANGLE = defaultdict(
    int,
    {
        frozenset([6, 5, 4]): 1,
        frozenset([4, 5, 3]): 2,
        frozenset([4, 3, 2]): 3,
        frozenset([3, 2, 1]): 4,
        frozenset([6, 8, 5]): 5,
        frozenset([5, 3, 13]): 6,
        frozenset([3, 13, 1]): 7,
        frozenset([8, 5, 10]): 8,
        frozenset([5, 10, 13]): 9,
        frozenset([13, 1, 14]): 10,
        frozenset([8, 9, 10]): 11,
        frozenset([10, 11, 13]): 12,
        frozenset([11, 13, 14]): 13,
        frozenset([9, 10, 11]): 14,
    },
)
max_num_triangle: int = 14
COL_PADS = np.array([1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 13, 14])
UNIT_OF_MEASURE_COL = {
    "pmax": "mV",
    "negpmax": "mV",
    "tmax": "ns",
    "area": "mV $\\cdot$ ns",
    "rms": "mV",
}
np.random.seed(12345)


In [4]:
eval_df = pd.read_csv("./evaluation.csv", header = 0, index_col = False)

In [5]:
print(eval_df.describe())
eval_df.info()

                  Id        pmax[0]     negpmax[0]        area[0]  \
count  128500.000000  128500.000000  128500.000000  128500.000000   
mean    64249.500000       4.864014      -5.267998       4.339755   
std     37094.899133       1.487738      75.816506       3.474568   
min         0.000000       1.560321  -26279.846101      -0.720471   
25%     32124.750000       4.126158      -5.261179       2.389559   
50%     64249.500000       4.666739      -4.646919       3.364344   
75%     96374.250000       5.258218      -4.092627       5.108143   
max    128499.000000      19.300223      -1.398572     108.044066   

             tmax[0]         rms[0]        pmax[1]     negpmax[1]  \
count  128500.000000  128500.000000  128500.000000  128500.000000   
mean       97.551876       1.340326      16.363815      -9.332822   
std        59.396351       0.342305      16.578094      74.268609   
min         0.000000       0.328655       1.887503  -21424.542339   
25%        46.284928       1.0960

In [6]:
dev_df = pd.read_csv("./development.csv", header = 0, index_col = False)
pd.unique(dev_df.groupby(by = ["x", "y"]).size())

array([100], dtype=int64)

In [7]:
print(dev_df.set_index("x").index.unique())
print(dev_df.set_index("x").index.unique())

Index([200.0, 205.0, 210.0, 215.0, 220.0, 225.0, 230.0, 235.0, 240.0, 245.0,
       250.0, 255.0, 260.0, 265.0, 270.0, 275.0, 280.0, 285.0, 290.0, 295.0,
       300.0, 305.0, 310.0, 315.0, 320.0, 325.0, 330.0, 335.0, 340.0, 345.0,
       350.0, 355.0, 360.0, 365.0, 370.0, 375.0, 380.0, 385.0, 390.0, 395.0,
       400.0, 405.0, 410.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440.0, 445.0,
       450.0, 455.0, 460.0, 465.0, 470.0, 475.0, 480.0, 485.0, 490.0, 495.0,
       500.0, 505.0, 510.0, 515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545.0,
       550.0, 555.0, 560.0, 565.0, 570.0, 575.0, 580.0, 585.0, 590.0, 595.0,
       600.0],
      dtype='float64', name='x')
Index([200.0, 205.0, 210.0, 215.0, 220.0, 225.0, 230.0, 235.0, 240.0, 245.0,
       250.0, 255.0, 260.0, 265.0, 270.0, 275.0, 280.0, 285.0, 290.0, 295.0,
       300.0, 305.0, 310.0, 315.0, 320.0, 325.0, 330.0, 335.0, 340.0, 345.0,
       350.0, 355.0, 360.0, 365.0, 370.0, 375.0, 380.0, 385.0, 390.0, 395.0,
       400.0, 405.0, 410.0, 

In [10]:
print(dev_df.isna().sum().sum())

0
